In [1]:
import numpy as np
import pandas as pd

We will load the data now:

*   Consolidées et définitives 2013-2022:
https://odre.opendatasoft.com/explore/dataset/eco2mix-national-cons-def/information/?disjunctive.nature
*   Au pas quart d'heure 2022-2023:
https://odre.opendatasoft.com/explore/dataset/eco2mix-national-tr/information/?disjunctive.nature
*   European wholesale prices:
https://ember-climate.org/data-catalogue/european-wholesale-electricity-price-data/

In [2]:
# Transform MW quarterly hour to consolidated MWh per hour
# load data
from google.colab import drive
drive.mount('/content/gdrive')
df=pd.read_csv('gdrive/My Drive/Data_CFE/eco2mix-national-cons-def.csv', sep=";")
df=df.sort_values(by="Date et Heure")

# Drop columns by name
columns_to_drop = ['Ech. physiques (MW)', 'Ech. comm. Angleterre (MW)', 'Ech. comm. Espagne (MW)', 'Ech. comm. Italie (MW)', 'Ech. comm. Suisse (MW)', 'Ech. comm. Allemagne-Belgique (MW)',
                   'Gaz - Cogénération (MW)','Fioul - Autres (MW)','Gaz - TAC (MW)', 'Gaz - CCG (MW)', 'Gaz - Autres (MW)',
                   'Taux de CO2 (g/kWh)', 'Fioul - TAC (MW)', 'Fioul - Cogénération (MW)', 'Hydraulique - Lacs (MW)',
                   "Hydraulique - Fil de l'eau + éclusée (MW)", 'Hydraulique - STEP turbinage (MW)', 'Bioénergies - Déchets (MW)', 'Bioénergies - Biomasse (MW)', 'Bioénergies - Biogaz (MW)',
                   'Pompage (MW)', 'Prévision J-1 (MW)', 'Prévision J (MW)' ]
df = df.drop(columns=columns_to_drop)
to_keep = ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00",
           "00:30", "01:30", "02:30", "03:30", "04:30", "05:30", "06:30", "07:30", "08:30", "09:30", "10:30", "11:30", "12:30", "13:30", "14:30", "15:30", "16:30", "17:30", "18:30", "19:30", "20:30", "21:30", "22:30", "23:30"]
df = df[df["Heure"].isin(to_keep)]

df = df.reset_index(drop=True)
df = df.drop(df.index[0])
df = df.reset_index(drop=True)

df2=df

# Convert 'Heure' column to datetime
df2['Heure_bis'] = pd.to_datetime(df2['Heure'], format='%H:%M')

# Group by 'Date' and 'Hour', then sum the 'Consommation (MW)' values
hourly_df = df2.groupby(['Date', df['Heure_bis'].dt.hour])['Consommation (MW)', 'Fioul (MW)', 'Charbon (MW)', 'Gaz (MW)', 'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)', 'Bioénergies (MW)'].sum().reset_index()

# Convert 'Heure_bis' to the format '00:00'
hourly_df['Heure_bis'] = hourly_df['Heure_bis'].astype(str).str.zfill(2) + ":00"
# Rename the 'Heure_bis' column to 'Heure'
hourly_df.rename(columns={'Heure_bis': 'Heure'}, inplace=True)

hourly_df['Consommation (MW)'] = df['Consommation (MW)'] / 2
columns_to_divide_by_2=['Consommation (MW)', 'Fioul (MW)', 'Charbon (MW)', 'Gaz (MW)', 'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)', 'Bioénergies (MW)']
hourly_df[columns_to_divide_by_2]=hourly_df[columns_to_divide_by_2]/2

df= hourly_df

df = df.reset_index(drop=True)
df = df.drop(df.index[0])
df = df.reset_index(drop=True)

# Combine 'Date' and 'Heure' into a new column 'Datetime'
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Heure'])


Mounted at /content/gdrive


<ipython-input-2-f5abbc3a3ec5>:5: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('gdrive/My Drive/Data_CFE/eco2mix-national-cons-def.csv', sep=";")
<ipython-input-2-f5abbc3a3ec5>:29: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  hourly_df = df2.groupby(['Date', df['Heure_bis'].dt.hour])['Consommation (MW)', 'Fioul (MW)', 'Charbon (MW)', 'Gaz (MW)', 'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)', 'Bioénergies (MW)'].sum().reset_index()


In [3]:
# load df_target

df_target=pd.read_csv('gdrive/My Drive/Data_CFE/France.csv')
df_target=df_target.sort_values(by="Datetime (Local)")

# We merge now the two datasets:

# Convert the 'Datetime (Local)' column to datetime format in df_target
df_target['Datetime (Local)'] = pd.to_datetime(df_target['Datetime (Local)'])

# Create ID in both dataframes
df['ID'] = df['Datetime']
df_target['ID'] = df_target['Datetime (Local)']

# Join df and df_target on ID
result = pd.merge(df, df_target, on='ID', how='inner')

# We drop some recurrent variables
to_drop = ['ISO3 Code', 'ID', 'Datetime (UTC)'] #'Date', 'Heure']
result = result.drop(columns=to_drop)

In [16]:
print(result.iloc[63606]) #max 2987,78 on 4 April 2022 iloc=63606

Date                          2022-04-04
Heure                              08:00
Consommation (MW)                8204.25
Fioul (MW)                        1493.0
Charbon (MW)                       465.5
Gaz (MW)                          8495.5
Nucléaire (MW)                   32110.0
Eolien (MW)                       4408.5
Solaire (MW)                       720.0
Hydraulique (MW)                 16159.5
Bioénergies (MW)                  1019.5
Datetime             2022-04-04 08:00:00
Country                           France
Datetime (Local)     2022-04-04 08:00:00
Price (EUR/MWhe)                 2987.78
Series                                 0
IndexTime                          63606
Name: 63606, dtype: object


In [17]:
# We reduce the shape of our data set for time computations limits
df = result

df['Series'] = 0
df["IndexTime"] = df.index

In [18]:
# Save the two parts to CSV files
df.to_csv('gdrive/My Drive/Data_CFE/df.csv', index=False)

In [19]:
#test
#test=pd.read_csv('gdrive/My Drive/Data_CFE/df.csv')
#test.tail(5)

In [22]:
#print(test.iloc[63606]) #max 2987,78 on 4 April 2022 iloc=63606

Date                          2022-04-04
Heure                              08:00
Consommation (MW)                8204.25
Fioul (MW)                        1493.0
Charbon (MW)                       465.5
Gaz (MW)                          8495.5
Nucléaire (MW)                   32110.0
Eolien (MW)                       4408.5
Solaire (MW)                       720.0
Hydraulique (MW)                 16159.5
Bioénergies (MW)                  1019.5
Datetime             2022-04-04 08:00:00
Country                           France
Datetime (Local)     2022-04-04 08:00:00
Price (EUR/MWhe)                 2987.78
Series                                 0
IndexTime                          63606
Name: 63606, dtype: object
